In [84]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [164]:
elev_importance = pd.read_csv('../data/processed/stationgraph/elevator-importance.csv')
outages = pd.read_csv('../data/processed/total_elevator_outage_to_20210605.csv')
ridership = pd.read_csv('../data/processed/ridership_per_station.csv')
elev_line = pd.read_csv('../data/processed/stationgraph/elevator_to_line_dir_station.csv')
nearest = pd.read_csv('../data/processed/nearest/nearest_stations.csv')

In [165]:
elev_line.head()

,equipment_id,line,direction,station
0,EL122,6,manhattan,Pelham Bay Park
1,EL124,A,north,175 St
2,EL124,A,south,175 St
3,EL126,5,south,Harlem-125 St
4,EL126,4,south,Harlem-125 St


In [166]:
elev_importance.head()

,Station,Elevator,Importance,Perc. Importance,Betweenness
0,Harlem-125 St,EL125,6,100.0,9.000000
1,Harlem-125 St,EL126,6,100.0,21.000000
2,125 St (IRT Third Avenue Line),EL142,8,100.0,1776.000000
3,125 St (IRT Third Avenue Line),EL143,4,50.0,3209.498276
4,125 St (IRT Third Avenue Line),EL144,4,50.0,3745.881887


In [170]:
outages.dtypes

equipment         object
duration_delta    object
dtype: object

In [171]:
outages.duration_delta = pd.to_timedelta(outages.duration_delta)

In [172]:
tmp = elev_importance.merge(elev_line.drop(columns=['direction']),how='left',left_on='Elevator',right_on='equipment_id')

In [173]:
tmp[tmp.equipment_id.isna()]

,Station,Elevator,Importance,Perc. Importance,Betweenness,equipment_id,line,station
0,Harlem-125 St,EL125,6,100.000000,9.00000,NaN,NaN,NaN
7,125 St (IRT Third Avenue Line),EL142,8,100.000000,1776.00000,NaN,NaN,NaN
20,14 St - 8 Av,EL221,6,100.000000,1776.00000,NaN,NaN,NaN
27,14 St - Union Sq,EL217,10,100.000000,1776.00000,NaN,NaN,NaN
46,161 St - Yankee Stadium,EL131,6,100.000000,1776.00000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
580,Fulton St,EL723,4,28.571429,16772.19112,NaN,NaN,NaN
595,Fulton St,EL733X,0,0.000000,1776.00000,NaN,NaN,NaN
596,50 St,EL207X,2,100.000000,1776.00000,NaN,NaN,NaN
609,Lexington Av / 53 St and 51 St,EL203X,6,100.000000,1776.00000,NaN,NaN,NaN


In [174]:
avg_daily_ridership = ridership[ridership.datetime<'2020-01-01'].groupby(['STATION','LINENAME']).mean().reset_index().drop(columns=['Unnamed: 0'])

In [175]:
avg_daily_ridership[avg_daily_ridership.STATION == '125 ST']

,STATION,LINENAME,estimated_entries,estimated_exits
16,125 ST,1,510.739726,785.238356
17,125 ST,23,865.534247,525.561644
18,125 ST,456,1846.112329,2087.287671
19,125 ST,ACBD,2071.457534,2361.487671


In [176]:
crosswalk = pd.read_csv('../data/crosswalk/Master_crosswalk.csv')

In [177]:
crosswalk.loc[crosswalk.station_name.str.contains('Times') & crosswalk.equipments.notnull(),'equipments'] = 'EL231X,ES216,ES217,ES218,EL229,EL232,EL233,EL230,ES261X,ES262X,ES268X,ES267X,EL232X'

In [178]:
crosswalk.head()

,Unnamed: 0,station_name,clean_lines,station_lines,el_station_name,el_lines,equipments,gtfs_station_name,gtfs_lines,turnstile_station_name,turnstile_lines,gtfs_stop_id,lat,lon,turnstile_units
0,0,Astor Pl,46,4-6-6 Express,NaN,NaN,NaN,Astor Pl,"4,6,6X",ASTOR PL,6,636,40.730054,-73.991070,R160
1,1,Canal St,46,4-6-6 Express,Canal St,6,"EL324,EL325",Canal St,"4,6,6X",CANAL ST,JNQRZ6W,639,40.718803,-74.000193,"R118,R462,R463"
2,2,50th St,12,1-2,NaN,NaN,NaN,50 St,"1,2",50 ST,1,126,40.761728,-73.983849,R116
3,3,Bergen St,234,2-3-4,NaN,NaN,NaN,Bergen St,"2,3,4",BERGEN ST,23,236,40.680829,-73.975098,R058
4,4,Pennsylvania Ave,34,3-4,NaN,NaN,NaN,Pennsylvania Av,"2,3,4,5",PENNSYLVANIA AV,3,255,40.664635,-73.894895,R067


In [179]:
el_crosswalk = crosswalk[crosswalk.equipments.notnull()]

In [180]:
el_crosswalk

,Unnamed: 0,station_name,clean_lines,station_lines,el_station_name,el_lines,equipments,gtfs_station_name,gtfs_lines,turnstile_station_name,turnstile_lines,gtfs_stop_id,lat,lon,turnstile_units
1,1,Canal St,46,4-6-6 Express,Canal St,6,"EL324,EL325",Canal St,"4,6,6X",CANAL ST,JNQRZ6W,639,40.718803,-74.000193,"R118,R462,R463"
10,10,Delancey St - Essex St,JMZ,J-M-Z,Delancey St Essex St,F/J/M/Z,"ES327,ES328",Delancy St - Essex St,"J,Z,M",DELANCEY ST,FJMZ,M18,40.718315,-73.987437,R142
14,14,DeKalb Ave,BDNQR,B-D-N-Q-R,Dekalb Av,B/Q/R,"EL370,EL372,EL371,ES309,ES308",DeKalb Av,"B,D,N,W,Q,R",DEKALB AV,BDNQR,R30,40.690635,-73.981824,R099
18,18,Intervale Ave,25,2-5,Intervale Av,2/5,ES108,Intervale Av,"2,5",INTERVALE AV,25,218,40.822181,-73.896736,R407
23,23,Junction Blvd,7,7-7 Express,Junction Blvd,7,"EL427,EL426,EL425",Junction Blvd,"7,7X",JUNCTION BLVD,7,707,40.749145,-73.869527,R097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,519,Grand Central - 42nd St,456,4-5-6-6 Express,42 St - Grand Central,4/5/6/7,"EL204,ES207,ES209,ES610X,ES203,ES204,ES206,ES2...",Grand Central - 42 St,"4,5,5X,6,6X",GRD CNTRL-42 ST,4567S,631,40.751776,-73.976848,"R045,R046,R047,R048"
524,524,Broadway Junction,JZ,J-Z,Broadway Junction,A/C/J/L/Z/LIRR,"ES347,ES331,ES330",Broadway Jct,"J,Z",BROADWAY JCT,ACJLZ,J27,40.679498,-73.904512,R103
526,526,Delancey St - Essex St,JMZ,J-M-Z,Delancey St Essex St,F/J/M/Z,"ES327,ES328",Delancy St - Essex St,"J,Z,M",DELANCEY/ESSEX,FJMZ,M18,40.718315,-73.987437,R142
528,528,Lexington Ave - 59th St,456,4-5-6-6 Express,59 St,4/5/6,"ES213,ES211,ES214,ES212",59 St,"4,5,5X,6,6X",59 ST,456NQRW,629,40.762526,-73.967967,"R050,R051"


In [181]:
a = 'EL324,EL325'
a.split(',')

['EL324', 'EL325']

In [182]:
el_crosswalk['els'] = [z.split(',') for z in el_crosswalk.equipments]

/opt/conda/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [183]:
el_crosswalk

,Unnamed: 0,station_name,clean_lines,station_lines,el_station_name,el_lines,equipments,gtfs_station_name,gtfs_lines,turnstile_station_name,turnstile_lines,gtfs_stop_id,lat,lon,turnstile_units,els
1,1,Canal St,46,4-6-6 Express,Canal St,6,"EL324,EL325",Canal St,"4,6,6X",CANAL ST,JNQRZ6W,639,40.718803,-74.000193,"R118,R462,R463","[EL324, EL325]"
10,10,Delancey St - Essex St,JMZ,J-M-Z,Delancey St Essex St,F/J/M/Z,"ES327,ES328",Delancy St - Essex St,"J,Z,M",DELANCEY ST,FJMZ,M18,40.718315,-73.987437,R142,"[ES327, ES328]"
14,14,DeKalb Ave,BDNQR,B-D-N-Q-R,Dekalb Av,B/Q/R,"EL370,EL372,EL371,ES309,ES308",DeKalb Av,"B,D,N,W,Q,R",DEKALB AV,BDNQR,R30,40.690635,-73.981824,R099,"[EL370, EL372, EL371, ES309, ES308]"
18,18,Intervale Ave,25,2-5,Intervale Av,2/5,ES108,Intervale Av,"2,5",INTERVALE AV,25,218,40.822181,-73.896736,R407,[ES108]
23,23,Junction Blvd,7,7-7 Express,Junction Blvd,7,"EL427,EL426,EL425",Junction Blvd,"7,7X",JUNCTION BLVD,7,707,40.749145,-73.869527,R097,"[EL427, EL426, EL425]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,519,Grand Central - 42nd St,456,4-5-6-6 Express,42 St - Grand Central,4/5/6/7,"EL204,ES207,ES209,ES610X,ES203,ES204,ES206,ES2...",Grand Central - 42 St,"4,5,5X,6,6X",GRD CNTRL-42 ST,4567S,631,40.751776,-73.976848,"R045,R046,R047,R048","[EL204, ES207, ES209, ES610X, ES203, ES204, ES..."
524,524,Broadway Junction,JZ,J-Z,Broadway Junction,A/C/J/L/Z/LIRR,"ES347,ES331,ES330",Broadway Jct,"J,Z",BROADWAY JCT,ACJLZ,J27,40.679498,-73.904512,R103,"[ES347, ES331, ES330]"
526,526,Delancey St - Essex St,JMZ,J-M-Z,Delancey St Essex St,F/J/M/Z,"ES327,ES328",Delancy St - Essex St,"J,Z,M",DELANCEY/ESSEX,FJMZ,M18,40.718315,-73.987437,R142,"[ES327, ES328]"
528,528,Lexington Ave - 59th St,456,4-5-6-6 Express,59 St,4/5/6,"ES213,ES211,ES214,ES212",59 St,"4,5,5X,6,6X",59 ST,456NQRW,629,40.762526,-73.967967,"R050,R051","[ES213, ES211, ES214, ES212]"


In [184]:
els = el_crosswalk.apply(lambda x: pd.Series(x['els']), axis=1).stack().reset_index(level=1, drop=True)
els.name = 'els'

In [185]:
el_crosswalk = el_crosswalk.drop('els', axis=1).join(els)
el_crosswalk['els'] = pd.Series(el_crosswalk['els'], dtype=object)

In [186]:
crosswalk_elevators = set(el_crosswalk[el_crosswalk.els.str.contains('EL')].els)

In [187]:
elev_importance[elev_importance.Elevator == 'EL232X']

,Station,Elevator,Importance,Perc. Importance,Betweenness
191,Times Sq - 42 St,EL232X,0,0.0,68383.039544


In [188]:
el_crosswalk[el_crosswalk.els.isin(list(crosswalk_elevators - set(elev_importance.Elevator)))].station_name.unique()

array(['Grand Central - 42nd St', 'Woodside - 61st St',
       '42nd St - Bryant Pk', '68th St - Hunter College',
       'Herald Sq - 34th St', '168th St', '181st St', '191st St',
       'Briarwood - Van Wyck Blvd', 'Bedford Park Blvd', '57th St',
       'Hoyt - Schermerhorn Sts', 'Jay St - MetroTech', 'Court St',
       '190th St', 'Chambers St', 'Clark St', '86th St'], dtype=object)

In [189]:
len(set(elev_importance.Elevator))

282

In [192]:
nearest.head()

,destination,walking_dist
0,Cathedral Pkwy (110th St),14
1,Junction Blvd,5
2,111th St,10
3,Woodhaven Blvd,10
4,Central Park North (110th St),17


In [198]:
merged = el_crosswalk.merge(avg_daily_ridership,how='left', left_on=['turnstile_station_name','turnstile_lines'],right_on=['STATION','LINENAME']).merge(elev_importance,how='right',left_on=['els'],right_on=['Elevator'])

In [199]:
merged = merged.merge(nearest,how='left',left_on='station_name',right_on='destination')

In [201]:
merged.columns

Index(['Unnamed: 0', 'station_name', 'clean_lines', 'station_lines',
       'el_station_name', 'el_lines', 'equipments', 'gtfs_station_name',
       'gtfs_lines', 'turnstile_station_name', 'turnstile_lines',
       'gtfs_stop_id', 'lat', 'lon', 'turnstile_units', 'els', 'STATION',
       'LINENAME', 'estimated_entries', 'estimated_exits', 'Station',
       'Elevator', 'Importance', 'Perc. Importance', 'Betweenness',
       'destination', 'walking_dist'],
      dtype='object')

In [203]:
merged_elevator_stats = merged[['station_name', 'clean_lines', 'equipments', 
       'gtfs_stop_id', 'lat', 'lon', 'estimated_entries', 'estimated_exits',
       'Elevator', 'Importance', 'Perc. Importance', 'Betweenness',
       'destination', 'walking_dist']].rename(columns={'estimated_entries':'avg_daily_entries','estimated_exits':'avg_daily_exits'})

In [204]:
merged_elevator_stats.to_csv('../data/processed/stationgraph/elevator_importance_merged.csv',index=False)